In [1]:
import os
import fnmatch
import tensorflow as tf
#from tensorflow.keras.models import Sequential  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split


data_dir = 'Bilder'
file_list = os.listdir(data_dir)
image_paths = []
steering_angles = []
pattern = "*.png"
for filename in file_list:
    if fnmatch.fnmatch(filename, pattern):
        image_paths.append(os.path.join(data_dir,filename))
        angle = int(filename[-7:-4])  # AAA part of timestamp_AAA.jpg --> AAA is the angle in range 045 - 135
        steering_angles.append(angle)

X_train, X_valid, y_train, y_valid = train_test_split( image_paths, steering_angles, test_size=0.2)
print(f"Training data: {len(X_train)} Validation data: {len(X_valid)}")

Training data: 20 Validation data: 6


In [2]:
#!pip3 uninstall kerasy

In [6]:
def cnn_model():
    
    # Keras Sequntial API
    model = tf.keras.Sequential(name='Lenkwinkelerkennung') 
    
    # Convolution Layers
    # elu: Expenential Linear Unit, similar to leaky Relu
    #model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(100, 200, 3), activation='elu')) 
    model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu')) 
    model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu'))
    model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu'))
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    model.add(Dropout(0.2)) # more robustness
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    
    # Fully Connected Layers
    model.add(Flatten()) 
    model.add(Dropout(0.2)) # more robustness
    model.add(Dense(100, activation='elu')) 
    model.add(Dense(50, activation='elu')) 
    model.add(Dense(10, activation='elu'))

    # Output Layer: turning angle
    model.add(Dense(1)) 
    
    # since this is a regression problem not classification problem,
    # we use MSE (Mean Squared Error) as loss function
    optimizer = Adam(lr=1e-3) # lr is learning rate
    model.compile(loss='mse', optimizer=optimizer)
    
    return model

model = cnn_model()
print(model.summary())

Model: "Lenkwinkelerkennung"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 3, 20, 64)         27712     
_________________________________________________________________
dropout_6 (Dropout)          (None, 3, 20, 64)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1152)      

In [11]:
# saves the model weights after each epoch if the validation loss decreased
model_output_dir = "/home/pi/git/camp2code4car/weltbeherrschungscode/Stefan"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(model_output_dir,'lane_navigation_check.h5'), verbose=1, save_best_only=True)

history = model.fit(
    X_train,
    y_train,
    batch_size=100,
    epochs=10,
    verbose=1,
    validation_data=(X_valid,y_valid)
)

# always save model output as soon as model finishes training
model.save(os.path.join(model_output_dir,'lane_navigation_final.h5'))

AttributeError: 'str' object has no attribute 'shape'